# Quantum Walks for Graph Search

This notebook explores quantum walk algorithms on different graph structures
and benchmarks their search performance against classical random walks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import sys
sys.path.append('..')

from src.coined_walk import run_coined_walk
from src.continuous_walk import evolve_continuous_walk, sweep_evolution
from src.graph_utils import (
    build_complete_graph, build_hypercube, build_cycle,
    classical_hitting_time
)
from src.search import search_continuous_walk, benchmark_search
from src.plotting import (
    plot_walk_distribution, plot_search_probability,
    plot_continuous_walk_evolution, plot_scaling_comparison
)

## 1. Discrete-Time Coined Quantum Walk

We start with a coined quantum walk on a small position register
and observe the non-classical probability distribution.

In [ ]:
# coined walk with Hadamard coin on 4-qubit position register
result = run_coined_walk(n_position_qubits=4, n_steps=8,
                         coin_type='hadamard', shots=8192, seed=42)

print(f"Position register: {result['positions']}")
print(f"Peak probability: {result['probabilities'].max():.4f}")
print(f"Peak position: {result['positions'][np.argmax(result['probabilities'])]}")

plot_walk_distribution(result['positions'], result['probabilities'],
                       title='Coined Quantum Walk (Hadamard, 8 steps)')

## 2. Continuous-Time Walk on Different Graphs

Continuous-time quantum walks evolve under the graph adjacency
Hamiltonian. The spreading behavior depends on the graph topology.

In [ ]:
# continuous walk on a cycle graph
G_cycle = build_cycle(16)
times = np.linspace(0, 20, 200)
sweep = sweep_evolution(G_cycle, initial_node=0, time_range=times)

plot_continuous_walk_evolution(sweep['times'], sweep['prob_matrix'],
                               node_labels=[f'Node {i}' for i in range(8)])
print("Continuous walk on 16-node cycle: probability spreads ballistically")

In [ ]:
# compare walk dynamics on complete vs cycle graph
G_complete = build_complete_graph(16)
sweep_complete = sweep_evolution(G_complete, initial_node=0, time_range=times)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i in range(min(8, 16)):
    axes[0].plot(times, sweep['prob_matrix'][:, i], alpha=0.7)
axes[0].set_title('Cycle Graph (16 nodes)')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Probability')

for i in range(min(8, 16)):
    axes[1].plot(times, sweep_complete['prob_matrix'][:, i], alpha=0.7)
axes[1].set_title('Complete Graph (16 nodes)')
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Probability')

plt.tight_layout()
plt.savefig('../results/walk_comparison.png', dpi=150)
plt.show()

## 3. Quantum Walk Search

Using a continuous-time quantum walk to search for a marked vertex.
On a complete graph, optimal search time scales as O(sqrt(N)).

In [ ]:
# search on complete graph with 64 nodes
G = build_complete_graph(64)
search_result = search_continuous_walk(G, marked_vertex=0)

print(f"Optimal time: {search_result['optimal_time']:.2f}")
print(f"Max success probability: {search_result['max_probability']:.4f}")
print(f"Expected sqrt(N) scaling: {np.sqrt(64) * np.pi/2:.2f}")

plot_search_probability(search_result['times'], search_result['success_probs'],
                        optimal_time=search_result['optimal_time'])

## 4. Scaling Comparison

Benchmark quantum walk search vs classical random walk across graph sizes.

In [ ]:
node_counts = [8, 16, 32, 64, 128]

# quantum search times
print("Quantum walk search:")
quantum_results = benchmark_search(build_complete_graph, node_counts)

# classical hitting times
print("\nClassical random walk:")
classical_times = []
for n in node_counts:
    G = build_complete_graph(n)
    ht = classical_hitting_time(G, start=1, target=0, n_trials=5000)
    classical_times.append(ht)
    print(f"  n={n}: {ht:.1f} steps")

plot_scaling_comparison(node_counts, quantum_results['optimal_times'],
                        classical_times)

In [ ]:
# summary table
print(f"{'Nodes':<8} {'Classical':<12} {'Quantum':<12} {'Speedup':<10}")
print('-' * 42)
for i, n in enumerate(node_counts):
    speedup = classical_times[i] / quantum_results['optimal_times'][i]
    print(f"{n:<8} {classical_times[i]:<12.1f} {quantum_results['optimal_times'][i]:<12.2f} {speedup:<10.1f}x")